# args & validate dataset

In [1]:
model_path = "E:\\data\\models\\gpt-boost\\gpt-tiny-memory"
data_path = "E:\\data\\corpus\\gpt-boost\\novel.txt"
tokenizer_path = ".\\tokenizer"
tb_path = ".\\runs\\gpt-tiny-memory"

In [2]:
n_embd=512
n_head=8
n_layer=4
n_positions=256
vocab_size=20000
finetuning_mode=False

In [3]:
learning_rate=1e-3
gradient_accumulation_steps=4
num_train_epochs=40
per_device_train_batch_size=8

# 1. create tokenizer

In [4]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained(tokenizer_path, max_len=256)

Special tokens have been added in the vocabulary, make sure the associated word emebedding are fine-tuned or trained.


# 2. initialize model

In [ ]:
import sys
sys.path.append("../")
from models.memory.modeling_gpt2_memory import GPT2LMHeadModel

if finetuning_mode:
    model = GPT2LMHeadModel.from_pretrained(model_path)
else:
    from transformers import GPT2Config
    config = GPT2Config(
        n_embd=n_embd,
        n_head=n_head,
        n_layer=n_layer,
        n_positions=n_positions,
        vocab_size=vocab_size
    )
    model = GPT2LMHeadModel(config=config)

FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.


In [ ]:
model.num_parameters()
# => 161405952 parameters (约1.6B)

# 3. build training Dataset

In [ ]:
from transformers import TextDataset

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=data_path,
    block_size=256,
)

Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# 4. initialize our Trainer & Train & Save

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=model_path,
    overwrite_output_dir=True,
    no_cuda=False,
    do_train=True,
    fp16=True,
    logging_dir=tb_path,
    logging_steps=100,
    learning_rate=learning_rate,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    save_steps=1000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [ ]:
trainer.train()

### 🎉 Save final model (+ tokenizer + config) to disk

In [ ]:
trainer.save_model(model_path)

#### ！！！注意，这里需要把词表拷贝到模型文件夹！！！

# 5. Check the model

In [ ]:
from transformers import pipeline

generate = pipeline(
    "text-generation",
    model=model_path,
)

In [ ]:
import time
time0 = time.time()
result = generate("""第八十四章 满月的呓语
　　克莱恩刚披上双排扣长礼服，拿起半高丝绸礼帽，往门口走去，忽然听见了层层回荡的虚幻祈求声。
　　谁？他微皱眉头，侧耳倾听了一下，但只能确认祈求者是一位女士，而且嗓音断断续续，似乎蕴藏着极大的痛苦。
　　想着也没什么特别紧要的事情，新晋“魔术师”克莱恩随手一扔，让半高丝绸礼帽又准确无误地挂到了衣帽架上，自身则返回卧室，逆走四步，进入巍峨雄伟的宫殿。
　　这一次，""", max_length=256, repetition_penalty=1, do_sample=True, top_k=40, temperature=1.2, num_return_sequences=3)
print(time.time()-time0)
for item in result:
  print(item["generated_text"])
  print()
  print("="*50)
  print()